# 아는 만큼 보이고 보이는 만큼 개선할 수 있다.

### “장애 인지와 분석에 대한 솔루션”

과제 1에서 만든 로그 분석기의 기능 스펙 이외에도 

다양한 관점에서 장애 원인 분석을 할 수 있는 오픈소스 도구들이 있을 것이다. 

어떠한 도구 라도 좋으니 분석을 하기 위한 방법과 장애가 났을 때 빠르게 상황을 인지할 수 있는 방법을 찾아 솔루션으로 제시하고 왜 그러한 방법을 솔루션으로 사용했는지 이유를 설명해보자.

 문제를 처음 받고 다양한 관점에서 여러 관점의 혼란이 올거라는 것이 시사되었다. 그러므로 나는 질문을 던져나가며 이 문제를 풀어 나갈까 한다.

다시한번, 다양한 시각에서 시작했기 때문에, end_point가 될 수 있는 시작점도 없을 것이라 판단했다. 그러므로, 문제에 관해서 질문을 던지고 그 질문을 해결해 나가는 과정에서 제가 생각하는 이상적인 솔루션을 나아가보겠습니다.

1. 문제에서 말하는 `장애 인지와 분석`은 누구를 위한 것인가?

개발자를 위한 것인가? 마케터를 위한 것인가?

개발자를 위한다면 서버의 트래픽을 모니터링해주는 서비스가 될 수 있고, 구글애널러틱스와 같은 서비스를 활용하여 고객의 행동을 분석하고, 특정 부분을 장애라고 인지할 수도 있다고 생각한다.

또는 최고통치권자를 위한 것 일 수도?

2. 1번문제에서 주어진 `Input date`에는 문제가 없는가?

    1. 분명 input date에는 STATUS CODE가 400, 500이면서 응답시간이 200~300초인 로그들이 존재하는것을 파악했다. => 문제가 있다.  

    2. 또한 로깅파일이 분산되어 있는 것도 잠재적인 문제가 있을 것이라 생각한다. API 서버가 여러 대일 경우, 각각의 API의 서버들이 어떤 일련의 동작으로 로깅이라는 동작을 실행할텐데, 분석 또는 상황을 인지하는 입장에서는 각각의 서버들의 로깅 모니터링해야 한다면 `tail -f ap1.daouoffice.com_access_2018-08-27.log` 이런식으로 현재 3개의 터미널을 띄어놓고 작업을 수행해야될지도 모른다라고 생각한다.

그래서 알아보니, 로깅을 중앙화하여(Centralized Logging) 일련의 쿼리를 통해 분석을 할 수 있는 도구들이 존재함을 알았다.
    3. 어떤 이유인지는 모르겠으나, 로깅의 포맷 중 user-agent가 없는 로그들이 빈번히 보였다. 해당 부분 포맷의 바로 뒤에 user-agnet가 없을 시, User-agent를 백업하듯 디바이스의 정보가 담겨져 있었지만, user-agent가 없다는 것은 개인적으로 status가 200이라고 해도 신뢰할 수 있는 정보인가에 대해 고려해볼 필요가 있다고 판단되었다. 다음은 User-agnet의 rfc7231 관련 내용이다.

```text
    https://tools.ietf.org/html/rfc7231#section-5.5.3
    A user agent SHOULD send a User-Agent field in each request
   unless specifically configured not to do so.
   
   A sender SHOULD limit generated product identifiers to what is
   necessary to identify the product; a sender MUST NOT generate
   advertising or other nonessential information within the product
   identifier.  A sender SHOULD NOT generate information in
   product-version that is not a version identifier (i.e., successive
   versions of the same product name ought to differ only in the
   product-version portion of the product identifier).
   
```

    내용을 살펴보다가 다시 로그로 돌아와보니 GO-Android/2.5.0.3 와 같은 product-version에 관련된 정보가 보이기도 했다. 내가 생각하는 것이 맞는가에 대해 다시한번 생각해볼 필요가 있다,

    4. 응답은 200이지만 응답시간이 터무니없이 높을 경우(230, 300) 등 이는 장애라고 말할수 있는가?

    이 부분에 대해서는 자세히 로그를 살펴보니, 파일 다운로드 등의 FIle io 등 오래 걸리는 것이라 문제되지 않았다.

    5. 특정 크롤링봇으로 판단되는 요소들이 존재하는 것을 발견했다.
    특정 크롤링봇에 대해서도 block을 해야될 것으로 판단된다.